In [2]:
!pip install --upgrade jupyter ipywidgets widgetsnbextension

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.2 MB 2.8 MB/s eta 0:00:01
   ---------------------------- ----------- 1.6/2.2 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 4.6 MB/s  0:00:00
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/12.3 MB 7.7 MB/s eta 0:00:02
   ------------ --------------------------- 3.9/12.3 MB 9.8 MB/s eta 0:00:01
   ---------------------- ----------------- 6.8/12.3 MB 11.3 MB/s eta 0:00:01
   --------------------------------- ------ 10.2/12.3 MB 12.5 MB/s eta 0:00:01
   ---------------------------------------- 12.3/12.3 MB 13.1 MB/s  0:00:00
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ----------------- ---------------------- 4.5/10.2 MB 20.7 MB/s eta 0:00:01
   --------------------------- ------------ 7.1/10.2 MB 16.8 MB/s eta 0:00:01
   --------------------

In [1]:
# 除了像之前使用 AutoModel 根据 checkpoint 自动加载模型以外，我们也可以直接使用模型对应的 Model 类，例如 BERT 对应的就是 BertModel：

from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")

注意，在大部分情况下，我们都应该使用 AutoModel 来加载模型。这样如果我们想要使用另一个模型（比如把 BERT 换成 RoBERTa），只需修改 checkpoint，其他代码可以保持不变。

所有存储在 HuggingFace Model Hub 上的模型都可以通过 Model.from_pretrained() 来加载权重，参数可以像上面一样是 checkpoint 的名称，也可以是本地路径（预先下载的模型目录），例如：

In [4]:
from transformers import BertModel

#model = BertModel.from_pretrained("./models/bert") #  not wo

In [5]:
# 保存模型通过调用 Model.save_pretrained() 函数实现，例如保存加载的 BERT 模型：

from transformers import AutoModel

model = AutoModel.from_pretrained("bert-base-cased")
model.save_pretrained("./models/bert-base-cased/")

这会在保存路径下创建两个文件：

config.json：模型配置文件，存储模型结构参数，例如 Transformer 层数、特征空间维度等；
pytorch_model.bin：又称为 state dictionary，存储模型的权重。
简单来说，配置文件记录模型的结构，模型权重记录模型的参数，这两个文件缺一不可。我们自己保存的模型同样通过 Model.from_pretrained() 加载，只需要传递保存目录的路径。

由于神经网络模型不能直接处理文本，因此我们需要先将文本转换为数字，这个过程被称为编码 (Encoding)，其包含两个步骤：

使用分词器 (tokenizer) 将文本按词、子词、字符切分为 tokens；
将所有的 token 映射到对应的 token ID。么

In [6]:
#分词器的加载与保存与模型相似，使用 Tokenizer.from_pretrained() 和 Tokenizer.save_pretrained() 函数。例如加载并保存 BERT 模型的分词器：

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
tokenizer.save_pretrained("./models/bert-base-cased/")



('./models/bert-base-cased/tokenizer_config.json',
 './models/bert-base-cased/special_tokens_map.json',
 './models/bert-base-cased/vocab.txt',
 './models/bert-base-cased/added_tokens.json')

In [ ]:
# 同样地，我们可以用 AutoModelForSequenceClassification 来加载预训练模型：
from transformers import AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenizer.save_pretrained("./models/bert-base-cased/")

('./models/bert-base-cased/tokenizer_config.json',
 './models/bert-base-cased/special_tokens_map.json',
 './models/bert-base-cased/vocab.txt',
 './models/bert-base-cased/added_tokens.json',
 './models/bert-base-cased/tokenizer.json')


#调用 Tokenizer.save_pretrained() 函数会在保存路径下创建三个文件：

special_tokens_map.json：映射文件，里面包含 unknown token 等特殊字符的映射关系；
tokenizer_config.json：分词器配置文件，存储构建分词器需要的参数；
vocab.txt：词表，一行一个 token，行号就是对应的 token ID（从 0 开始）。

In [2]:
# 编码与解码文本
# 前面说过，文本编码 (Encoding) 过程包含两个步骤：

# 分词：使用分词器按某种策略将文本切分为 tokens；
# 映射：将 tokens 转化为对应的 token IDs。
# 下面我们首先使用 BERT 分词器来对文本进行分词：

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


In [3]:
#'using', 'a', 'transform', '##er', 'network', 'is', 'simple']
#可以看到，BERT 分词器采用的是子词切分策略，它会不断切分词语直到获得词表中的 token，例如 “transformer” 会被切分为 “transform” 和 “##er”。

#然后，我们通过 convert_tokens_to_ids() 将切分出的 tokens 转换为对应的 token IDs：

ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

[7993, 170, 13809, 23763, 2443, 1110, 3014]


In [4]:
# 还可以通过 encode() 函数将这两个步骤合并，并且 encode() 会自动添加模型需要的特殊 token，例如 BERT 分词器会分别在序列的首尾添加 

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
sequence_ids = tokenizer.encode(sequence)

print(sequence_ids)

[101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102]


In [5]:
# 文本解码 (Decoding) 与编码相反，负责将 token IDs 转换回原来的字符串。注意，解码过程不是简单地将 token IDs 映射回 tokens，还需要合并那些被分为多个 token 的单词。下面我们通过 decode() 函数解码前面生成的 token IDs：

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

decoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])
print(decoded_string)

decoded_string = tokenizer.decode([101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102])
print(decoded_string)

Using a transformer network is simple
[CLS] Using a Transformer network is simple [SEP]


In [ ]:
# 现实场景中，我们往往会同时处理多段文本，而且模型也只接受批 (batch) 数据作为输入，即使只有一段文本，也需要将它组成一个只包含一个样本的 batch，例如：

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor([ids])
print("Input IDs:\n", input_ids)

output = model(input_ids)
print("Logits:\n", output.logits)


Input IDs:
 tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits:
 tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


In [7]:
# 注意，上面的代码仅作为演示。实际场景中，我们应该直接使用分词器对文本进行处理，例如对于上面的例子：

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer  = AutoTokenizer.from_pretrained(checkpoint)
model      = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence   = "I've been waiting for a HuggingFace course my whole life."

tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print("Inputs Keys:\n", tokenized_inputs.keys())
print("\nInput IDs:\n", tokenized_inputs["input_ids"])

output = model(**tokenized_inputs)
print("\nLogits:\n", output.logits)

Inputs Keys:
 KeysView({'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])})

Input IDs:
 tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])

Logits:
 tensor([[-1.5607,  1.6123]], grad_fn=<AddmmBackward0>)


In [14]:
# 按批输入多段文本产生的一个直接问题就是：batch 中的文本有长有短，而输入张量必须是严格的二维矩形，维度为 
# ，即每一段文本编码后的 token IDs 数量必须一样多。例如下面的 ID 列表是无法转换为张量的：

batched_ids = [
    [200, 200, 200],
    [200, 200]
]

In [15]:
padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]

In [16]:
# 因此，在进行 Padding 操作时，我们必须明确告知模型哪些 token 是我们填充的，它们不应该参与编码。这就需要使用到 Attention Mask 了，在前面的例子中相信你已经多次见过它了。

# Attention Mask
# Attention Mask 是一个尺寸与 input IDs 完全相同，且仅由 0 和 1 组成的张量，0 表示对应位置的 token 是填充符，不参与计算。当然，一些特殊的模型结构也会借助 Attention Mask 来遮蔽掉指定的 tokens。

# 对于上面的例子，如果我们通过 attention_mask 标出填充的 padding token 的位置，计算结果就不会有问题了：

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]
batched_attention_masks = [
    [1, 1, 1],
    [1, 1, 0],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
outputs = model(
    torch.tensor(batched_ids), 
    attention_mask=torch.tensor(batched_attention_masks))
print(outputs.logits)

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


In [8]:
# 直接使用分词器
# 正如前面所说，在实际使用时，我们应该直接使用分词器来完成包括分词、转换 token IDs、Padding、构建 Attention Mask、截断等操作。例如：

from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequences = [
    "I've been waiting for a HuggingFace course my whole life.", 
    "So have I!"
]

model_inputs = tokenizer(sequences)
print(model_inputs)

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [18]:
# 实际使用分词器时，我们通常会同时进行 padding 操作和截断操作，并设置返回格式为 Pytorch 张量，这样就可以直接将分词结果送入模型：

from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequences = [
    "I've been waiting for a HuggingFace course my whole life.", 
    "So have I!"
]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
print(tokens)
output = model(**tokens)
print(output.logits)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  2061,  2031,  1045,   999,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
tensor([[-1.5607,  1.6123],
        [-3.6183,  3.9137]], grad_fn=<AddmmBackward0>)


In [9]:
# <!-- 编码句子对
# 除了对单段文本进行编码以外（batch 只是并行地编码多个单段文本），对于 BERT 等包含“句子对”预训练任务的模型，它们的分词器都支持对“句子对”进行编码，例如： -->

from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

inputs = tokenizer("This is the first sentence.", "This is the second one.")
print(inputs)

tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"])
print(tokens)

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']


In [10]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sentence1_list = ["First sentence.", "This is the second sentence.", "Third one."]
sentence2_list = ["First sentence is short.", "The second sentence is very very very long.", "ok."]

tokens = tokenizer(
    sentence1_list,
    sentence2_list,
    padding=True,
    truncation=True,
    return_tensors="pt"
)
print(tokens)
print(tokens['input_ids'].shape)

{'input_ids': tensor([[ 101, 2034, 6251, 1012,  102, 2034, 6251, 2003, 2460, 1012,  102,    0,
            0,    0,    0,    0,    0,    0],
        [ 101, 2023, 2003, 1996, 2117, 6251, 1012,  102, 1996, 2117, 6251, 2003,
         2200, 2200, 2200, 2146, 1012,  102],
        [ 101, 2353, 2028, 1012,  102, 7929, 1012,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
torch.Size([3, 18])


In [22]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch

# Load the model and tokenizer
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name)

# Prepare input
text = "This is a test sentence."
inputs = tokenizer(text, return_tensors="pt")

# Access the underlying DistilBertModel and its embeddings
distilbert_model = model.distilbert
embeddings_layer = distilbert_model.embeddings

# Get the token embeddings
input_ids = inputs["input_ids"]
token_embeddings = embeddings_layer(input_ids)

print(token_embeddings.shape)

c:\Users\zhen-\anaconda3\envs\torch_cuda\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\zhen-\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at dis

torch.Size([1, 8, 768])


分词器还可以通过 return_tensors 参数指定返回的张量格式：
* 设为 pt 则返回 PyTorch 张量；
* 设为 tf 则返回 TensorFlow 张量，
* 设为 np 则返回 NumPy 数组。例如：

In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequences = [
    "I've been waiting for a HuggingFace course my whole life.", 
    "So have I!"
]

model_inputs = tokenizer(sequences, padding=True, return_tensors="pt") # PyTorch tensors
print(model_inputs)

model_inputs = tokenizer(sequences, padding=True, return_tensors="np") # NumPy arrays
print(model_inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  2061,  2031,  1045,   999,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
{'input_ids': array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  2061,  2031,  1045,   999,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


编码句子对
除了对单段文本进行编码以外（batch 只是并行地编码多个单段文本），对于 BERT 等包含“句子对”预训练任务的模型，它们的分词器都支持对“句子对”进行编码，例如：


In [12]:

from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

inputs = tokenizer("This is the first sentence.", "This is the second one.")
print(inputs)

tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"])
print(tokens)

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']


+ [CLS] (Classification Token) 
+ [SEP] (Separator Token)

5.4 添加 Token
实际操作中，我们还经常会遇到输入中需要包含特殊标记符的情况，例如使用 
 和 
 标记出文本中的实体。由于这些自定义 token 并不在预训练模型原来的词表中，因此直接运用分词器处理就会出现问题。

例如直接使用 BERT 分词器处理下面的句子：


In [13]:

from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sentence = 'Two [ENT_START] cars [ENT_END] collided in a [ENT_START] tunnel [ENT_END] this morning.'
print(tokenizer.tokenize(sentence))

['two', '[', 'en', '##t', '_', 'start', ']', 'cars', '[', 'en', '##t', '_', 'end', ']', 'collided', 'in', 'a', '[', 'en', '##t', '_', 'start', ']', 'tunnel', '[', 'en', '##t', '_', 'end', ']', 'this', 'morning', '.']


In [14]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# 假设你已经加载了 tokenizer 和 model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

descriptions = ['start of entity', 'end of entity']

# 确保模型在 eval 模式（避免 dropout 等影响）
model.eval()

with torch.no_grad():
    for i, token in enumerate(reversed(descriptions), start=1):
        tokenized = tokenizer.tokenize(token)
        print(f"Tokenized '{token}': {tokenized}")
        
        if not tokenized:
            print(f"Warning: '{token}' tokenized to empty list!")
            continue
            
        tokenized_ids = tokenizer.convert_tokens_to_ids(tokenized)
        # 获取底层 DistilBERT 模型的 embeddings
        word_embeddings = model.distilbert.embeddings.word_embeddings.weight
        new_embedding = word_embeddings[tokenized_ids].mean(dim=0)
        
        # 将新嵌入写入特殊 token 位置（如 [unused1], [unused2] 等）
        # 注意：DistilBERT 的 vocab 末尾通常有特殊 token（如 [unused999]）
        # 你需要确保 -i 位置是可写的（最好使用预留的特殊 token）
        model.distilbert.embeddings.word_embeddings.weight[-i, :] = new_embedding.clone().detach()

# 查看最后两个 token 的嵌入
print("Last two embeddings:")
print(model.distilbert.embeddings.word_embeddings.weight[-2:, :])

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenized 'end of entity': ['end', 'of', 'entity']
Tokenized 'start of entity': ['start', 'of', 'entity']
Last two embeddings:
tensor([[-0.0490, -0.0195, -0.0482,  ..., -0.0171,  0.0313, -0.0294],
        [-0.0146, -0.0251, -0.0270,  ..., -0.0267,  0.0213, -0.0392]],
       grad_fn=<SliceBackward0>)


初始化为已有 token 的值
更为高级的做法是根据新添加 token 的语义来进行初始化。例如将值初始化为 token 语义描述中所有 token 的平均值，假设新 token 
 的语义描述为 
，那么初始化 
 的 embedding 为： 
 

这里 
 表示预训练模型的 embedding 矩阵。对于上面的例子，我们可以分别为 
 和 
 编写对应的描述，然后再对它们的值进行初始化：
